# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

v = 0
for sqrt_value in generator:
  v+=sqrt_value
  print(sqrt_value)
print(f"The sum for limit 5 is : {v}")

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
The sum for limit 5 is : 8.382332347441762


In [8]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

v = 0
for i in generator:
  print(i)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [ ]:
!pip install "dlt[duckdb]"

In [5]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='city_dataset')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

print(info)


Pipeline dlt_colab_kernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset city_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708169942.130331 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset city_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708169942.9796815 is LOADED and contains no failed jobs


In [9]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)


# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708169942.130331,lHvmy0r2HTD4LQ,None
1,2,Person_2,27,City_A,1708169942.130331,9Ia2PRRHi5m93Q,None
2,3,Person_3,28,City_A,1708169942.130331,YQ4Y3amPn1rW7w,None
3,4,Person_4,29,City_A,1708169942.130331,zLgpimx8FyoMVQ,None
4,5,Person_5,30,City_A,1708169942.130331,radndoY0D/b88A,None
5,3,Person_3,33,City_B,1708169942.9796815,Fru4PPXX2dJHDQ,Job_3
6,4,Person_4,34,City_B,1708169942.9796815,rbzIrHRb/jEq8Q,Job_4
7,5,Person_5,35,City_B,1708169942.9796815,X1FTvI36S8QmbQ,Job_5
8,6,Person_6,36,City_B,1708169942.9796815,q6ZGqorw7LhElw,Job_6
9,7,Person_7,37,City_B,1708169942.9796815,mIpHTxs8EGuUWA,Job_7


In [10]:
people.age.sum()

353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [11]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [20]:
# to do: homework :)
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='city_dataset')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="data",
										write_disposition="merge",
                    primary_key="id")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)




Pipeline dlt_colab_kernel_launcher load step completed in 0.45 seconds
1 load package(s) were loaded to destination duckdb and into dataset city_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708171488.8480842 is LOADED and contains no failed jobs


In [21]:
# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="data",
										write_disposition="merge",
                    primary_key="id")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset city_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708171492.8880064 is LOADED and contains no failed jobs


In [22]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

merged_data = conn.sql("SELECT * FROM data").df()
display(merged_data)


# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ data                │
│ people              │
│ people_1            │
│ people_2            │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708171488.8480842,j0cne+keyR1FwA,None
1,2,Person_2,27,City_A,1708171488.8480842,c0xyAORFAEADDg,None
2,5,Person_5,35,City_B,1708171492.8880064,GPJ9j1icIfhflg,Job_5
3,7,Person_7,37,City_B,1708171492.8880064,qRHSLvTf0rki9g,Job_7
4,3,Person_3,33,City_B,1708171492.8880064,NlYtRTyQDAOmyA,Job_3
5,6,Person_6,36,City_B,1708171492.8880064,XpQo2sOLvIimrQ,Job_6
6,8,Person_8,38,City_B,1708171492.8880064,WWs7GCzPYlVbCg,Job_8
7,4,Person_4,34,City_B,1708171492.8880064,ddyjxjvhwdpYoA,Job_4


In [23]:
merged_data.age.sum()

266

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX